In [59]:
#https://github.com/urchade/GLiNER/tree/main/examples
import tkinter as tk
from tkinter import scrolledtext
import spacy
from spacy.training.example import Example

In [60]:
#https://icd.who.int/browse11
#dictionary for disease names(keys) to medical codes(values). 
disease_to_code = {
    "type 2 diabetes mellitus": "E11",
    "asthma": "J45",
    "viral hepatitis": "B18",
    "chronic kidney failure": "N18",
    "essential hypertension": "I10",
    "acute myocardial infarction": "I21",
    "osteoarthritis of the knee": "M17",
    "major depressive disorder": "F32",
    "anemia due to chronic disease": "D63",
    "malignant neoplasm of the lung": "C34"
}

In [61]:
TRAIN_DATA = []
for disease, code in disease_to_code.items():
    text = f"The patient has been diagnosed with {disease}."
    start = text.find(disease)
    end = start + len(disease)
    TRAIN_DATA.append((text, {"entities": [(start, end, "Disease")]}))

In [62]:
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")
ner.add_label("Disease")
optimizer = nlp.begin_training()
for i in range(20):
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5)

In [63]:
#Saves the trained model and print Prints a success message.
nlp.to_disk("disease_ner_model")
print("Model trained and saved successfully.")

Model trained and saved successfully.


In [64]:
#https://github.com/riyajha2305/Healthcare-Diagnosis-Chatbot-MS-Hackathon
# Function to process user input and display results in GUI 
nlp = spacy.load("disease_ner_model")
def process_input():
    result_text.delete(1.0, tk.END)
    user_input = input_text.get("1.0", tk.END).strip()
    if not user_input:
        result_text.insert(tk.END, "Please enter a valid sentence.")
        return
    doc = nlp(user_input)
    found_diseases = False
    for ent in doc.ents:
        if ent.label_ == "Disease":
            found_diseases = True
            disease = ent.text
            code = disease_to_code.get(disease, "Unknown code")
            result_text.insert(tk.END, f"Disease: {disease}, Code: {code}\n")
    if not found_diseases:
        result_text.insert(tk.END, "No diseases detected in the input.")

In [65]:
#the main window
root = tk.Tk()
root.title("Medical Coding Recognition Tool")
root.geometry("600x400")
root.configure(bg="#f0f8ff")
title_label = tk.Label(root, text="Medical Coding Recognition Tool", font=("Arial", 18, "bold"), bg="#f0f8ff", fg="#005a9c")
title_label.pack(pady=10)
instructions_label = tk.Label(root, text="Enter a sentence describing a disease:", font=("Arial", 12), bg="#f0f8ff", fg="#333333")
instructions_label.pack()
input_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=5, font=("Arial", 12))
input_text.pack(pady=10)
process_button = tk.Button(root, text="Recognize Diseases", command=process_input, font=("Arial", 12, "bold"), bg="#005a9c", fg="white")
process_button.pack(pady=10)
result_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=8, font=("Arial", 12), state='normal')
result_text.pack(pady=10)
root.mainloop()